In [4]:
import pandas as pd
import os 
import os.path as osp
import glob

In [9]:
os.getcwd()

'D:\\Project- Electro\\social_behavior_electro\\data_organization\\python'

In [12]:
# folder_name = 'D://Project- Electro//social_behavior_electro//analysis//affiliative//population analysis results'
folder_name = osp.join('..','..','analysis','affiliative','population analysis results')
df_dict_list = glob.glob(osp.join(folder_name, 'Population analysis results for LFP - Mean power*'))
df_dict_list

['..\\..\\analysis\\affiliative\\population analysis results\\Population analysis results for LFP - Mean power_aff_111_13.xlsx',
 '..\\..\\analysis\\affiliative\\population analysis results\\Population analysis results for LFP - Mean power_aff_112_2.xlsx',
 '..\\..\\analysis\\affiliative\\population analysis results\\Population analysis results for LFP - Mean power_aff_12-15.xlsx',
 '..\\..\\analysis\\affiliative\\population analysis results\\Population analysis results for LFP - Mean power_aff_14-16.xlsx']

In [61]:
def fix_file_paths(df_files):
    
    df_files = df_files.rename(columns={df_files.columns[0]:'timestamps', df_files.columns[0]:'filename', })
    
    return df_files


In [154]:
substrings_to_remove = ['Mean','Before','After', 'During','First', 'Second', 'Area']

df_all_list = []
for df_name in df_dict_list:
    df_dict = pd.read_excel(df_name, None)

    for sheet, df in df_dict.items():
#         if len(df.columns) > 1:
#             continue
        if "All" in sheet:
            continue
            
        if "Uploaded" in sheet:
            filenames = fix_file_paths(df)
            filenames = filenames. iloc[:,0]
            max_len = len(filenames)
            continue
        
        
        df_temp = pd.DataFrame()
        area_name = sheet
        for sub_string_to_remove in substrings_to_remove:
            area_name = area_name.replace(sub_string_to_remove,'')
        area_name = str(area_name)

        vals = df.iloc[:max_len]
        
        if "Before" in sheet:
            stage = "1_Before"
        elif "During" in sheet:
            stage = "2_During"
        elif "After" in sheet:
            stage = "3_After"
        else:
            print(sheet)
            raise
            
        if "First" in sheet:
            freq = '4-12'
        elif "Second" in sheet:
            freq = "30-80"
        else:
            raise
        
        df_temp['values'] = vals
        df_temp['area'] = area_name
        df_temp['stage'] = stage
        df_temp['freq'] = freq
        df_temp['filenames'] = filenames
        
        df_all_list.append(df_temp)
df_all = pd.concat(df_all_list)          

In [155]:
df = df_all.copy()
# Create a pivot table to unstack 'freq' and 'stage' columns
pivot_df = pd.pivot_table(df, values='values', index=['filenames'], columns=['area','freq','stage'])

# Reset the index to flatten the columns
pivot_df = pivot_df.reset_index()
pivot_df = pivot_df.set_index(('filenames',"",""))
pivot_df = pivot_df.rename_axis(index = {('filenames',"",""):'filenames'})

In [156]:
# Function to calculate the differences and add columns
def add_difference_columns(df):
    areas = df.columns.get_level_values('area').unique()
    freqs = df.columns.get_level_values('freq').unique()

    for area in areas:
        for freq in freqs:
            
            before_col = (area, freq, '1_Before')
            during_col = (area, freq, '2_During')
            after_col = (area, freq, '3_After')
            
            diff_during_before = df[during_col] - df[before_col]
            diff_after_before = df[after_col] - df[before_col]

            diff_col_name_during_before = (area,freq,'4_DiffDuringBefore')
            diff_col_name_after_before =  (area,freq,'5_DiffAfterBefore')

            df[diff_col_name_during_before] = diff_during_before
            df[diff_col_name_after_before] = diff_after_before

    return df

# Call the function to add difference columns
df_with_diff = add_difference_columns(pivot_df)


In [158]:
def sort_columns(df):
    # Sort the columns based on the specified rule
    sorted_columns = sorted(df.columns, key=lambda col: (col[0], col[1], col[2]))

    # Reindex the DataFrame with the sorted columns
    sorted_df = df.reindex(columns=sorted_columns)

    return sorted_df


df_with_diff_sorted = sort_columns(df_with_diff)
df_with_diff_sorted

area                                                     111           \
freq                                                   30-80            
stage                                               1_Before 2_During   
filenames                                                               
Chamber_Rat22-probe21-Day1-Behavior_and_Optogen...       NaN      NaN   
Chamber_Rat26-Probe25-Day1-Behavior_and_Optogen...   3.61410   4.7430   
Chamber_Rat26-Probe25-Day4-Behavior_and_Optogen...   1.94540   3.4285   
Chamber_Rat5_Probe8_day1_Behavior_and_Optogenet...  13.90330  13.7319   
Chamber_Rat8-Probe12_day1_Behavior_and_Optogene...       NaN      NaN   
Chamber_Rat8-probe8-Day2-sniffing_Behavior_and_...   9.77240  10.3846   
chamber_Rat1-probe4-day2-Behavior_and_Optogenet...       NaN      NaN   
chamber_Rat12-probe16-day3_Behavior_and_Optogen...       NaN      NaN   
chamber_Rat14-probe14-day2-Behavior_and_Optogen...   7.80120   9.0122   
chamber_Rat14-probe14-day3-Behavior_and_Optogen...   7.30890   8.5279   
chamber_Rat15-probe17-day1-Behavior_and_Optogen...   4.54180   5.3859   
chamber_Rat16-probe18-day3-Behavior_and_Optogen...   8.85080  10.9108   
chamber_Rat19-Probe18-day3-Behavior_and_Optogen...   7.61000   8.4979   
chamber_Rat19-Probe20-day2-Behavior_and_Optogen...  10.86660  12.0645   
chamber_Rat19-probe18-Day2-Behavior_and_Optogen...   7.96320   7.9079   
chamber_Rat19-probe18-day6-Behavior_and_Optogen...   8.29840   9.9184   
chamber_Rat19-probe20-day1-Behavior_and_Optogen...  10.71460  10.8939   
chamber_Rat2-probe1-Sniffing-day1-Behavior_and_...   0.36111   3.5951   
chamber_Rat2-probe1-sniffing-day2-Behavior_and_...  -0.64745   1.6739   
chamber_Rat2-probe1-sniffing-day3-Behavior_and_...   6.72580   8.5885   
chamber_Rat20-Probe19-day1-Behavior_and_Optogen...   5.51210   7.0254   
chamber_Rat21-probe20-day1-Behavior_and_Optogen...   5.91220   7.9850   
chamber_Rat23-probe22-Day2-Behavior_and_Optogen...  10.76370  11.2898   
chamber_Rat3-Probe2-sniffing-Day3-Behavior_and_...   5.86820   7.4229   
chamber_Rat3-probe2-sniffing-day2-Behavior_and_...   5.62220   7.1540   
chamber_Rat4-probe3-sniffing-day1-Behavior_and_...   3.86100   6.9382   
chamber_Rat5_probe8_day2_Behavior_and_Optogenet...  12.87800  13.2477   
chamber_Rat5_probe8_day4_Behavior_and_Optogenet...  12.13560  13.1591   
chamber_Rat8-Probe8-Day1-sniffing_Behavior_and_...   9.25900  10.0314   
chamber_Rat8-probe12-day2_Behavior_and_Optogene...       NaN      NaN   
chamber_Rat8-probe12-day3_Behavior_and_Optogene...   0.92369   2.0392   
chamber_Rat9-probe11-day2_Behavior_and_Optogene...  12.17850  12.2949   
chamber_rat14-probe14-day 1-Behavior_and_Optoge...  11.50460  11.6041   
chamber_rat9-probe11-day1_Behavior_and_Optogene...  12.27050  12.3552   

area                                                          \
freq                                                           
stage                                                3_After   
filenames                                                      
Chamber_Rat22-probe21-Day1-Behavior_and_Optogen...       NaN   
Chamber_Rat26-Probe25-Day1-Behavior_and_Optogen...   3.68000   
Chamber_Rat26-Probe25-Day4-Behavior_and_Optogen...   2.39750   
Chamber_Rat5_Probe8_day1_Behavior_and_Optogenet...  13.94320   
Chamber_Rat8-Probe12_day1_Behavior_and_Optogene...       NaN   
Chamber_Rat8-probe8-Day2-sniffing_Behavior_and_...   9.46520   
chamber_Rat1-probe4-day2-Behavior_and_Optogenet...       NaN   
chamber_Rat12-probe16-day3_Behavior_and_Optogen...       NaN   
chamber_Rat14-probe14-day2-Behavior_and_Optogen...   8.65570   
chamber_Rat14-probe14-day3-Behavior_and_Optogen...   7.53090   
chamber_Rat15-probe17-day1-Behavior_and_Optogen...   5.04740   
chamber_Rat16-probe18-day3-Behavior_and_Optogen...   9.81430   
chamber_Rat19-Probe18-day3-Behavior_and_Optogen...   7.42980   
chamber_Rat19-Probe20-day2-Behavior_and_Optogen...  11.52340   
chamber_Rat19-probe18-Day2-Behavior_and_Optogen...   6.89780   
chamber_R

In [159]:
# save files
df_with_diff_sorted.to_excel(osp.join(folder_name,'lfp_summary.xlsx'))